In [5]:
import warnings
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer, transpile, IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
import pandas as pd
import math

from qiskit.quantum_info import Statevector

from scipy.stats import chi2_contingency, ttest_ind, levene

import unittest
import hypothesis.strategies as st
from hypothesis import given, settings

pi = np.pi

In [11]:
def measure_z(circuit, qubit_indexes):
    cBitIndex = 0
    for index in qubit_indexes:
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

def measure_x(circuit, qubitIndexes):
    cBitIndex = 0
    for index in qubitIndexes:
        circuit.h(index)
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

def measure_y(circuit, qubit_indexes):
    cBitIndex = 0
    for index in qubit_indexes:
        circuit.sdg(index)
        circuit.h(index)
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

In [169]:
# Completed but more testing required
def assertEntangled(backend,qc,qubits_to_assert,measurements_to_make,alpha = 0.05):
    # makes sure qubits_to_assert is a list
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register add them according to length of qubit list
    if (qc.num_clbits == 0):
        qc.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (len(qubits_to_assert) != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")
    
    zQuantumCircuit = measure_z(qc, qubits_to_assert)
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    q1=[]
    q2=[]

    classicalQubitIndex = 1
    for qubit in qubits_to_assert:
        for measurement in zMemory:
            if (measurement[2-classicalQubitIndex] == '0'):
                if(qubit==qubits_to_assert[0]):
                    q1.append(measurement[2-classicalQubitIndex])
                else:
                    q2.append(measurement[2-classicalQubitIndex])
            else:
                if(qubit==qubits_to_assert[0]):
                    q1.append(measurement[2-classicalQubitIndex])
                else:
                    q2.append(measurement[2-classicalQubitIndex])                   
        classicalQubitIndex+=1

    measDict = dict.fromkeys(['qubit1','qubit2'])
    measDict = {'qubit1': q1,'qubit2':q2}
    measDf = pd.DataFrame.from_dict(measDict)
    ct = pd.crosstab(measDf.qubit1,measDf.qubit2)

    chiVal, pVal, dOfFreedom, exp = chi2_contingency(ct)
    print("chi square value: ",chiVal,"p value: ",pVal,"expected values: ",exp)
    if(pVal>alpha):
        raise(AssertionError("states are not entangled"))
    else:
        print("states are entangled")
        
backend = Aer.get_backend('aer_simulator')
qc3 = QuantumCircuit(2)
qc3.x(1)
# qc3.x(1)
# qc3.x(0)
qc3.h(0)
qc3.cnot(0,1)
print(qc3)
assertEntangled(backend,qc3,[0,1],200,0.05)

     ┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
chi square value:  196.01643280573975 p value:  1.545888708011713e-44 expected values:  [[49.955 47.045]
 [53.045 49.955]]
states are entangled


In [26]:
def getDf(qc,qubits_to_assert,measurements_to_make,backend):

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register add them according to length of qubit list
    if (qc.num_clbits == 0):
        qc.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (len(qubits_to_assert) != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")

    ## divide measurements to make by 3 as we need to run measurements twice, one for x and one for y
    measurements_to_make = measurements_to_make // 3
    yQuantumCircuit = measure_y(qc.copy(), qubits_to_assert)
    xQuantumCircuit = measure_x(qc.copy(), qubits_to_assert)
    zQuantumCircuit = measure_z(qc, qubits_to_assert)

    yJob = execute(yQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    yMemory = yJob.result().get_memory()
    yCounts = yJob.result().get_counts()

        ## get x axis results
    xJob = execute(xQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    xMemory = xJob.result().get_memory()
    xCounts = xJob.result().get_counts()
    ## get z axis results
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    zCounts = zJob.result().get_counts()

    resDf = pd.DataFrame(columns=['0','1','+','i','-','-i'])
    classical_qubit_index = 1
    for qubit in qubits_to_assert:
        zero_amount, one_amount, plus_amount, i_amount, minus_amount, minus_i_amount = 0,0,0,0,0,0
        for experiment in xCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                plus_amount += xCounts[experiment]
            else:
                minus_amount += xCounts[experiment]
        for experiment in yCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                i_amount += yCounts[experiment]
            else:
                minus_i_amount += yCounts[experiment]
        for experiment in zCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                zero_amount += zCounts[experiment]
            else:
                one_amount += zCounts[experiment]
        df = {'0':zero_amount, '1':one_amount,
              '+':plus_amount, 'i':i_amount,
              '-':minus_amount,'-i':minus_i_amount}

        resDf = resDf.append(df, ignore_index = True)
        classical_qubit_index+=1

    resDf['+'] = resDf['+'].astype(int)
    resDf['i'] = resDf['i'].astype(int)
    resDf['-'] = resDf['-'].astype(int)
    resDf['-i'] = resDf['-i'].astype(int)
    resDf['0'] = resDf['0'].astype(int)
    resDf['1'] = resDf['1'].astype(int)
    return resDf


In [36]:
# Completed but more testing required
## assert that qubits are equal
def assertEqual(backend, quantumCircuit, qubits_to_assert, measurements_to_make, alpha):
    ## needs to make at least 2 measurements, one for x axis, one for y axis
    ## realistically we need more for any statistical significance
    if (measurements_to_make < 2):
        raise ValueError("Must make at least 2 measurements")

    # makes sure qubits_to_assert is a list
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    resDf1 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)
    resDf2 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)
    resDf3 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)
    resDf4 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)
    resDf5 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)

    q1Vals = []
    q2Vals = []
    q1Vals.extend([resDf1.at[0,'1'],resDf1.at[0,'-'],resDf1.at[0,'-i'], resDf2.at[0,'1'],resDf2.at[0,'-'],resDf2.at[0,'-i'], resDf3.at[0,'1'],resDf3.at[0,'-'],resDf3.at[0,'-i'],  resDf4.at[0,'1'],resDf4.at[0,'-'],resDf4.at[0,'-i'], resDf5.at[0,'1'],resDf5.at[0,'-'],resDf5.at[0,'-i']])
    print(q1Vals)
    q2Vals.extend([resDf1.at[1,'1'],resDf1.at[1,'-'],resDf1.at[1,'-i'], resDf2.at[1,'1'],resDf2.at[1,'-'],resDf2.at[1,'-i'],resDf3.at[1,'1'],resDf3.at[1,'-'],resDf3.at[1,'-i'],resDf4.at[1,'1'],resDf4.at[1,'-'],resDf4.at[1,'-i'],resDf5.at[1,'1'],resDf5.at[1,'-'],resDf5.at[1,'-i']  ])
    print(q2Vals)

    tTest, pValue = ttest_ind(q1Vals, q2Vals, alternative = 'two-sided') # Apply t test
    print("stat: ",tTest, "pValue: ", pValue)
    if pValue > alpha:
        print("The two qubits are equal (fail to reject null hypothesis) ")
    else:
        print("There is a significant difference between the two qubits (reject null hypothesis)")

qc = QuantumCircuit(2)
backend = Aer.get_backend('aer_simulator')
# qc.initialize([0, 1/np.sqrt(2), -1.j/np.sqrt(2), 0], qc.qubits)
qc.h(0)
# qc.cnot(0,1)
qc.x(1)
qc.p(0.5*2*math.pi/100, 1)
# qc.h(1)
# qc.p(10*2*math.pi/100, 0)
# qc.p(20*2*math.pi/100, 1)
assertEqual(backend, qc, [0,1], 300000, 0.05)
# assertEqual(backend, qc, [0,0], 300000, 0.05)

       0       1       +      i      -     -i
0  50195   49805  100000  49983      0  50017
1      0  100000   50097  49894  49903  50106
Second Measurement:
       0       1      +      i      -     -i
0  49968   50032  50217  49723  49783  50277
1      0  100000  49850  50085  50150  49915
[49805, 0, 50017, 50032, 49783, 50277, 50174, 49917, 50059, 49923, 50072, 49987, 49951, 49894, 49900]
[100000, 49903, 50106, 100000, 50150, 49915, 100000, 49900, 50112, 100000, 50144, 49922, 100000, 50228, 50108]
stat:  -2.8150435498340123 pValue:  0.008828946191117312
There is a significant difference between the two qubits (reject null hypothesis)


In [80]:
# Complete needs to be checked

# Assertion to check if expected probability of viewing a particular qubitstate is equal to its actual probaility
# Qubitchoice is an optional argument that if passed will only compare the expected probabilty with the probability of observing
# that particular qubit (first or second) in the desired qubitState for the qc 
def assertProbability(qc, qubitState :str, expectedProbability, qubitChoice=None):
    sv = Statevector.from_label("00") # Creates a statevector with states 00
    evl = sv.evolve(qc) # Passes the qc into the statevector in order to evolve 
    # Performs a check to observe if qubitChoice has been passed or not 
    if(qubitChoice!=None):
        probs = evl.probabilities_dict([qubitChoice]) # If passed we will get the probabilities for that particular qubit
    else:
        probs = evl.probabilities_dict()
    probsRound = {key: round(values,2) for key,values in probs.items()} # rounds off the probabilities in the dictionary

    # Loops over the prob dictionary with rounded values 
    for key,value in probsRound.items():
        if(key==qubitState):
            if(value==expectedProbability):
                print("Expected Probability present")
                return True
            else:
                raise(AssertionError("Probability not present"))
    raise(AssertionError("Probability not present or Desired state has no probability"))
            

qc = QuantumCircuit(2)
qc.h(0)
assertProbability(qc,"0",1,1)
print("------")
assertProbability(qc,"00",0.5)

Expected Probability present
------
Expected Probability present


True